In [6]:
import numpy as np
import math
import matplotlib.pyplot as plt  
from sklearn.metrics import mean_squared_error
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Bidirectional
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

import sys
import xlwt
import MySQLdb
from sklearn.preprocessing import MinMaxScaler


def transfromData(trainRaw, testRaw,windosSize):  ##Train ratial, train, test
    sc = MinMaxScaler(feature_range = (0, 1))
    #npRaw = np.array(rawData)
    #scaledData = sc.fit_transform(npRaw.reshape(-1,1))  
    #trainData = np.array(scaledData[:trainCount]).reshape(-1,1)##train 
    #testData = np.array(scaledData[-testCount:]).reshape(-1,1)
    
    npTrain = sc.fit_transform(np.array(trainRaw).reshape(-1,1))
    npTest = sc.fit_transform(np.array(testRaw).reshape(-1,1))
    

    X_train, y_train = splitXy(npTrain,windosSize)
    X_test, y_test = splitXy(npTest,windosSize)
    return sc, X_train, y_train, X_test, y_test

def splitXy(data,windosSize):
    windows = []
    for i in range(windosSize, data.shape[0]):
        windows.append(data[i-windosSize:i, 0])
    np.random.shuffle(windows)
    X = []
    y = []
    for i in range(len(windows)):
        X.append(windows[i][:6])
        y.append(windows[i][-(windosSize-6):])
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X,y
def buildModel(outSize):
    regressor = Sequential()
    #regressor.add(Bidirectional(LSTM(units=50,return_sequences=True),input_shape = (X_train.shape[1], 1)))
    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50,return_sequences=True))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units = outSize))
    # Compiling
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return regressor
def getStationList(cursor):
    stationList = []
    queryString = "SELECT distinct station from cleandata"
    cursor.execute(queryString)
    results = cursor.fetchall()
    for row in results :
        val = str(row).strip("',()")
        stationList.append(val)
    stationList.remove('富貴角')##only 106少數資料
    return stationList
def Visualize():
    predicted = sc.inverse_transform(regressor.predict(X_test))
    originY = sc.inverse_transform (y_test)
    print("MSE : ["+str(mean_squared_error(predicted, originY))+"]")
    # Visualising the results
    plt.plot(originY[:100], color = 'red', label = 'Real')  
    plt.plot(predicted[:100], color = 'blue', label = 'Predicted ') 
    plt.legend()
    plt.show()
def writeExcelHead(sheet1,epochs,station):
    sheet1.write(0,1,station)
    raw = 1
    for e in range(epochs):
        sheet1.write(raw,0,e+1)
        raw+=1

##get data##

def fetchData(cursor,station,windosSize):
    queryString = "SELECT * from cleandata where station='"+station+"' and item = 'PM2.5' and date NOT like '2017%'"
    cursor.execute(queryString)
    results = cursor.fetchall()

    trainRawData = []
    for row in results:
        for i in range(3,27):
            trainRawData.append(float(row[i]))
            
    queryString = "SELECT * from cleandata where station='"+station+"' and item = 'PM2.5' and date like '2017%'"
    cursor.execute(queryString)
    results = cursor.fetchall()

    testRawData = []
    for row in results:
        for i in range(3,27):
            testRawData.append(float(row[i]))

    sc, X_train, y_train, X_test, y_test = transfromData(trainRawData,testRawData,windosSize)
    return sc, X_train, y_train, X_test, y_test
def train(regressor,sc, X_train, y_train, X_test, y_test,epochs):
    for i in range(epochs):
        regressor.fit(X_train, y_train,validation_split=0.2, epochs = 1, batch_size = 32,verbose=2)
        predicted = sc.inverse_transform(regressor.predict(X_test))
        originY = sc.inverse_transform (y_test)
        mse = mean_squared_error(predicted, originY)
        print("Epoch : " +str(i)+", MSE : ["+str(mse)+"]")
        print('-------------------------------------------')
  
    return mse

In [7]:
import datetime
db=MySQLdb.connect(host="localhost",user="root", passwd="swater0", db="airdb", charset="utf8")
cursor = db.cursor()


epochs = 20

stationList = getStationList(cursor)

col=1
for station in stationList:
    print("training : " +station)
    MSEs = []
    for windowSize in range(7,17):
        sc, X_train, y_train, X_test, y_test = fetchData(cursor,station,windowSize)
        regressor = buildModel(windowSize-6)
        mse = train(regressor,sc,X_train, y_train, X_test, y_test,epochs)
        MSEs.append(mse)
        
    book = xlwt.Workbook(encoding="utf-8")
    sheet1 = book.add_sheet("Sheet1")
    writeExcelHead(sheet1,epochs,station)
    row = 1
    for m in MSEs:
        sheet1.write(row,1,m)
        row+=1
    book.save("excelFiles/LSTM/LSTMresult"+station+".xls")
        
    print('check point at ' + str(datetime.datetime.now()))

db.close()





training : 二林
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 8s - loss: 0.0031 - val_loss: 0.0021
Epoch : 0, MSE : [67.121008035305]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 6s - loss: 0.0014 - val_loss: 8.3441e-04
Epoch : 1, MSE : [26.209321933794012]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 6s - loss: 9.7915e-04 - val_loss: 8.0966e-04
Epoch : 2, MSE : [27.376134165018936]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 9.3589e-04 - val_loss: 7.2826e-04
Epoch : 3, MSE : [26.23450853296214]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 8.8177e-04 - val_loss: 7.5140e-04
Epoch : 4, MSE : [27.13823305316961]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1

Epoch : 4, MSE : [54.43269219167098]
-------------------------------------------
Train on 17138 samples, validate on 4285 samples
Epoch 1/1


KeyboardInterrupt: 

In [25]:


book.save("Test1EXCCCC.xls")